<a href="https://colab.research.google.com/github/allan-gon/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

#Imports

In [5]:
import numpy as np

from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score
                             )
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

#Cleaning

In [6]:
df.drop(['EASE-MENT','APARTMENT_NUMBER','ADDRESS'],axis=1,inplace=True)

df.dropna(inplace=True)

def remove_comma(cell):
  return cell.replace(",","")

df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].apply(remove_comma)

def make_int(cell):
  if cell.isdigit():
    return int(cell)
  else:
    return np.NaN

df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].apply(remove_comma)

#Masking

In [7]:
useable = df[
             (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
             ((df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000))
             ].copy()
useable.drop("BUILDING_CLASS_CATEGORY",axis=1,inplace=True)

In [8]:
useable['SALE_DATE'] = pd.to_datetime(useable['SALE_DATE'])

#One Hot Encoding

In [9]:
column_trans = make_column_transformer(
    (OneHotEncoder(),['NEIGHBORHOOD','BOROUGH',"TAX_CLASS_AT_PRESENT" ,
                      'BUILDING_CLASS_AT_PRESENT','BUILDING_CLASS_AT_TIME_OF_SALE']),
                      remainder='passthrough'
)

ridge = Ridge(normalize=True)

pipe = make_pipeline(column_trans, ridge)

#TTS

In [10]:
train = useable[useable['SALE_DATE'].dt.month < 4]

X_train = train.drop(['SALE_PRICE','SALE_DATE'],axis=1)
y_train = train['SALE_PRICE']

test = useable[useable['SALE_DATE'].dt.month == 4]

X_test = test.drop(["SALE_PRICE","SALE_DATE"],axis=1)
y_test = test["SALE_PRICE"]

#Baseline

In [11]:
baseline = y_train.mean()

#Predict

In [12]:
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['NEIGHBORHOOD', 'BOROUGH',
                                                   'TAX_CLASS_AT_PRESENT',
                                                   'BUILDING_CLASS_AT_PRESENT',
                              

In [13]:
y_known = pipe.predict(X_train)
y_new = pipe.predict(X_test)

In [14]:
mae = mean_absolute_error(y_train,y_known)
rmse = np.sqrt(mean_squared_error(y_train, y_known))
r_sqr = r2_score(y_train, y_known)

print(f"Train MAE: {mae}")
print(f"Train RMSE: {rmse}")
print(f"Train r_sqr: {r_sqr}")

Train MAE: 166029.40384136105
Train RMSE: 232007.0602490939
Train r_sqr: 0.3667443001215934


In [15]:
mae = mean_absolute_error(y_test,y_new)
rmse = np.sqrt(mean_squared_error(y_test, y_new))
r_sqr = r2_score(y_test, y_new)

print(f"Test MAE: {mae}")
print(f"Test RMSE: {rmse}")
print(f"Test r_sqr: {r_sqr}")

Test MAE: 167590.72089037535
Test RMSE: 243450.37310455588
Test r_sqr: 0.32695901463238686


Test MAE: 167648.2359908375
Test RMSE: 244097.98911293977
Test R^2: 0.32337346366523617

In [16]:
base_mae = mean_absolute_error(y_test, [baseline] * len(y_test))

print(f"If we always guessed the price to be the average price we'd be off by {base_mae}")
print(f"but by using our model we'be off by {mae}")

If we always guessed the price to be the average price we'd be off by 211564.20053849227
but by using our model we'be off by 167590.72089037535


#Select K Best

In [17]:
#max is 9
'''
for i in range(1,9):
  selector = SelectKBest(k=i)

  X_train_selected = selector.fit_transform(X_train.select_dtypes(exclude=object),y_train)
  X_test_selected = selector.transform(X_test.select_dtypes(exclude=object))

  mask = selector.get_support()

  rd = Ridge()

  rd.fit(X_train[X_train.select_dtypes(exclude=object).columns[mask]],y_train)

  y_hat = rd.predict(X_test[X_test.select_dtypes(exclude=object).columns[mask]])

  mae = mean_absolute_error(y_test,y_hat)
  rmse = np.sqrt(mean_squared_error(y_test, y_hat))
  r_sqr = r2_score(y_test, y_hat)

  print(f'This is with {i}')

  print(f"Train MAE: {mae}")
  print(f"Train RMSE: {rmse}")
  print(f"Train r_sqr: {r_sqr}\n\n\n")'''

'\nfor i in range(1,9):\n  selector = SelectKBest(k=i)\n\n  X_train_selected = selector.fit_transform(X_train.select_dtypes(exclude=object),y_train)\n  X_test_selected = selector.transform(X_test.select_dtypes(exclude=object))\n\n  mask = selector.get_support()\n\n  rd = Ridge()\n\n  rd.fit(X_train[X_train.select_dtypes(exclude=object).columns[mask]],y_train)\n\n  y_hat = rd.predict(X_test[X_test.select_dtypes(exclude=object).columns[mask]])\n\n  mae = mean_absolute_error(y_test,y_hat)\n  rmse = np.sqrt(mean_squared_error(y_test, y_hat))\n  r_sqr = r2_score(y_test, y_hat)\n\n  print(f\'This is with {i}\')\n\n  print(f"Train MAE: {mae}")\n  print(f"Train RMSE: {rmse}")\n  print(f"Train r_sqr: {r_sqr}\n\n\n")'